In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import gluonts
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.rolling_dataset import generate_rolling_dataset, StepStrategy
from gluonts.evaluation import make_evaluation_predictions, Evaluator
os.chdir('/home/reffert/DeepAR_InfluenzaForecast')
from PythonFiles.model import model, preprocessing, split_forecasts_by_week, plot_coverage
from PythonFiles.Configuration import Configuration
#from PythonFiles.neighbor import NeighborhoodListDataset
config = Configuration()
influenza_df = pd.read_csv("/home/reffert/DeepAR_InfluenzaForecast/Notebooks/DataProcessing/influenza.csv", sep=',')
population_df = pd.read_csv("/home/reffert/DeepAR_InfluenzaForecast/Notebooks/DataProcessing/PopulationVector.csv", sep=',')
neighbourhood_df = pd.read_csv("/home/reffert/DeepAR_InfluenzaForecast/Notebooks/DataProcessing/AdjacentMatrix.csv", sep=',')

In [ ]:
locations = list(influenza_df.location.unique())
#Process the df into a uniformly spaced df
spaced_df = influenza_df.loc[influenza_df.location.isin(locations), ['value', 'location', 'date']]
spaced_df = preprocessing(config, spaced_df, check_count=False, output_type="corrected_df")

#seperate the intervals for training and testing
train_set = spaced_df.loc[(spaced_df.index <= config.train_end_time) & (spaced_df.index >= config.train_start_time), :]
test_set = spaced_df.loc[(spaced_df.index >= config.train_start_time) & (spaced_df.index <= config.test_end_time), :]
start_time = min(test_set.index.difference(train_set.index))
end_time = max(test_set.index.difference(train_set.index))

train_data_with_neighborhood = PandasDatasetWithNeighborhood.from_long_dataframe(
    dataframe=train_set,
    item_id="location",
    target="value",
    freq=config.freq,
    #one_dim_target=True,
    neighborhood=neighbourhood_df,
)

#Format the train and test_set into a PandasDataset
train_set = PandasDataset.from_long_dataframe(dataframe=train_set, item_id='location', target="value", freq=config.freq)
#test_set = PandasDataset.from_long_dataframe(dataframe=test_set, item_id='location', target="value", freq=config.freq)

#test_set = generate_rolling_dataset(dataset=test_set,
 #                                   strategy=StepStrategy(prediction_length=4,step_size=1),
  #                                  start_time=pd.Period(start_time,config.freq),
   #                                 end_time=pd.Period(end_time,config.freq)
    #                                )
print(train_data_with_neighborhood) 

In [3]:
train_set = PandasDataset.from_long_dataframe(dataframe=train_set, item_id='location', target="value", freq=config.freq)

print(train_set)

PandasDataset(dataframes={'LK Ahrweiler':             value      location
2010-01-03    1.0  LK Ahrweiler
2010-01-10    6.0  LK Ahrweiler
2010-01-17    1.0  LK Ahrweiler
2010-01-24    2.0  LK Ahrweiler
2010-01-31    1.0  LK Ahrweiler
...           ...           ...
2017-12-03    0.0  LK Ahrweiler
2017-12-10    1.0  LK Ahrweiler
2017-12-17    0.0  LK Ahrweiler
2017-12-24    1.0  LK Ahrweiler
2017-12-31    2.0  LK Ahrweiler

[418 rows x 2 columns], 'LK Aichach-Friedberg':             value              location
2010-01-03    2.0  LK Aichach-Friedberg
2010-01-10    0.0  LK Aichach-Friedberg
2010-01-17    1.0  LK Aichach-Friedberg
2010-01-24    1.0  LK Aichach-Friedberg
2010-01-31    0.0  LK Aichach-Friedberg
...           ...                   ...
2017-12-03    0.0  LK Aichach-Friedberg
2017-12-10    0.0  LK Aichach-Friedberg
2017-12-17    0.0  LK Aichach-Friedberg
2017-12-24    0.0  LK Aichach-Friedberg
2017-12-31    0.0  LK Aichach-Friedberg

[418 rows x 2 columns], 'LK Alb-Donau-Kreis'